In [1]:
#dbutils.fs.mkdirs("/FileStore/tables/landing/")

In [2]:
pip install graphframes==3.1.3

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement graphframes==3.1.3 (from versions: 0.6)
ERROR: No matching distribution found for graphframes==3.1.3


In [3]:
import pkg_resources

try:
    version = pkg_resources.get_distribution("graphframes").version
    print("GraphFrames version:", version)
except pkg_resources.DistributionNotFound:
    print("GraphFrames is not installed.")


GraphFrames version: 0.6


In [4]:
#%fs ls dbfs:/FileStore/tables

In [5]:
#dbutils.fs.mkdirs("/FileStore/tables/on_time_flight_performance")

In [6]:
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.functions import *
from graphframes import *
from pyspark.sql import SparkSession

from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import DateType


In [7]:
#input files
tripdelaysFilePath = "data/data/departuredelays.csv"
airportsnaFilePath = "data/data/airport_codes_na.txt"
tripdelaysParquet  = "data/data/landing/tripdelays"
airportsnaParquet  = "data/data/landing/airportsna"

In [8]:
spark = SparkSession.builder.appName("SparkExample").config("spark.some.config.option", "config-value").getOrCreate()

In [9]:
#display(spark.read.csv(tripdelaysFilePath))
display(spark.read.option("delimiter", "\t").text(airportsnaFilePath))

DataFrame[value: string]

In [10]:
df = spark.read.option("header", "true").option("inferschema", "true").option("delimiter","\t").csv(airportsnaFilePath)
     
spark.read.option("header","true").option("inferschema","true").option("delimeter","\t").csv(tripdelaysFilePath).write.parquet(tripdelaysParquet,mode = 'overwrite')

In [11]:
df.show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



In [12]:
airporstna = spark.read.parquet(airportsnaParquet)
departuredelays = spark.read.parquet(tripdelaysParquet)


In [13]:
airporstna.createOrReplaceTempView("airporstna1")
departuredelays.createOrReplaceTempView("departuredelays1")

In [14]:
spark.sql("select * from airporstna1").show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
|   Alliance|   NE|    USA| AIA|
|     Alpena|   MI|    USA| APN|
|    Altoona|   PA|    USA| AOO|
|   Amarillo|   TX|    USA| AMA|
|Anahim Lake|   BC| Canada| YAA|
|  Anchorage|   AK|    USA| ANC|
|   Appleton|   WI|    USA| ATW|
|     Arviat|  NWT| Canada| YEK|
|  Asheville|   NC|    USA| AVL|
|      Aspen|   CO|    USA| ASE|
+-----------+-----+-------+----+
only showing top 20 rows



In [15]:
Iata = spark.sql("select distinct IATA from (select distinct(origin) as IATA from departuredelays1 union all select distinct(destination) as IATA from departuredelays1) a")
Iata.createOrReplaceTempView("Iata")

In [16]:
airports = airporstna.join(Iata,on = "IATA",how="inner").select("City","State","Country","IATA")


In [17]:
airports.show()
airports.createOrReplaceTempView("airports_df")

+-------------------+-----+-------+----+
|               City|State|Country|IATA|
+-------------------+-----+-------+----+
|International Falls|   MN|    USA| INL|
|        New Orleans|   LA|    USA| MSY|
|            Spokane|   WA|    USA| GEG|
|      Orange County|   CA|    USA| SNA|
|            Burbank|   CA|    USA| BUR|
|          Green Bay|   WI|    USA| GRB|
|        Great Falls|   MT|    USA| GTF|
|        Idaho Falls|   ID|    USA| IDA|
|       Grand Rapids|   MI|    USA| GRR|
|             Joplin|   MO|    USA| JLN|
|             Eugene|   OR|    USA| EUG|
|         Providence|   RI|    USA| PVD|
|         Greensboro|   NC|    USA| GSO|
|       Myrtle Beach|   SC|    USA| MYR|
|            Oakland|   CA|    USA| OAK|
|              Butte|   MT|    USA| BTM|
|               Cody|   WY|    USA| COD|
|              Fargo|   ND|    USA| FAR|
|         Fort Smith|   AR|    USA| FSM|
|          Marquette|   MI|    USA| MQT|
+-------------------+-----+-------+----+
only showing top

In [18]:
departuredelays_geo = spark.sql("""
    select
        cast(f.date as int) as tripid,
        concat("2014", concat("-", concat(concat(substr(cast(f.date as string),1,2),"-"),
        concat(substr(cast(f.date as string),3,2),
        concat(" ", concat(substr(cast(f.date as string),5,2),
        concat(":", concat(substr(cast(f.date as string),7,2),":00")))))))) as local_date,
        cast(f.delay as int),
        cast(f.distance as int),
        f.origin as src,
        f.destination as dst,
        o.city as src_city,
        d.city as dst_city,
        o.state as src_state,
        d.state as dst_state
    from departuredelays1 f
    join airports_df o on f.origin = o.iata
    inner join airports_df d on f.destination = d.iata
""")
departuredelays_geo.show()


+-------+------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
| tripid|        local_date|delay|distance|src|dst|   src_city|           dst_city|src_state|dst_state|
+-------+------------------+-----+--------+---+---+-----------+-------------------+---------+---------+
|3311115|2014-33-11 11:5:00|   -8|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|3302000|2014-33-02 00:0:00|  -10|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|3292000|2014-32-92 00:0:00|  -19|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|3281115|2014-32-81 11:5:00|   -7|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|3271115|2014-32-71 11:5:00|    9|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|3261115|2014-32-61 11:5:00|   -5|     221|MSP|INL|Minneapolis|International Falls|       MN|       MN|
|3251115|2014-32-51 11:5:00|   -4|     221|MSP|INL|Minneapolis|I

In [19]:
tripvertices = airports.withColumnRenamed("IATA","id").distinct()
tripEdges = departuredelays_geo.select("tripid","delay","src","dst","src_city","dst_city")

In [20]:
tripEdges.show()

+-------+-----+---+---+-----------+-------------------+
| tripid|delay|src|dst|   src_city|           dst_city|
+-------+-----+---+---+-----------+-------------------+
|3311115|   -8|MSP|INL|Minneapolis|International Falls|
|3302000|  -10|MSP|INL|Minneapolis|International Falls|
|3292000|  -19|MSP|INL|Minneapolis|International Falls|
|3281115|   -7|MSP|INL|Minneapolis|International Falls|
|3271115|    9|MSP|INL|Minneapolis|International Falls|
|3261115|   -5|MSP|INL|Minneapolis|International Falls|
|3251115|   -4|MSP|INL|Minneapolis|International Falls|
|3241115|   -9|MSP|INL|Minneapolis|International Falls|
|3232000|   -9|MSP|INL|Minneapolis|International Falls|
|3222000|  -10|MSP|INL|Minneapolis|International Falls|
|3211115|    0|MSP|INL|Minneapolis|International Falls|
|3201115|   -6|MSP|INL|Minneapolis|International Falls|
|3191115|   -5|MSP|INL|Minneapolis|International Falls|
|3181115|    2|MSP|INL|Minneapolis|International Falls|
|3171115|   25|MSP|INL|Minneapolis|International

In [21]:
from graphframes import GraphFrame

In [2]:
tripGraph = GraphFrame(tripvertices, tripEdges)

NameError: name 'GraphFrame' is not defined

In [23]:
print(f"Airports: {tripGraph.vertices.count()}")
print(f"Trips: {tripGraph.edges.count()}")

Airports: 279
Trips: 1361141


In [24]:
longestDelay = tripGraph.edges.groupBy().max("delay")

In [34]:
longestDelay.show()

+----------+
|max(delay)|
+----------+
|      1642|
+----------+



DataFrame[max(delay): int]

In [25]:
print(f'On-time / Early Flights: {tripGraph.edges.filter("delay <= 0").count()}')
print(f'Delayed Flights: {tripGraph.edges.filter("delay > 0").count()}')

On-time / Early Flights: 780469
Delayed Flights: 580672


In [26]:
# Delay can be <= 0 i.e. the flight left on time or early
sfoDelayedTrips = tripGraph.edges.\
  filter("src = 'SFO' and delay > 0").\
  groupBy("src", "dst").\
  avg("delay").\
  sort(desc("avg(delay)"))

In [27]:
display(sfoDelayedTrips)

DataFrame[src: string, dst: string, avg(delay): double]

In [28]:
# After displaying tripDelays, use Plot Options to set `state_dst` as a Key.
tripDelays = tripGraph.edges.filter("delay > 0")
display(tripDelays)

DataFrame[tripid: int, delay: int, src: string, dst: string, src_city: string, dst_city: string]

In [29]:
#(display(tripEdges))

In [30]:
# States with the longest cumulative delays (with individual delays > 100 minutes) (origin: Philadelphia)
#tripEdges.createOrReplaceTempView("tripgraph")
display(tripGraph.edges.filter("src ='PHL' and delay > 100"))

DataFrame[tripid: int, delay: int, src: string, dst: string, src_city: string, dst_city: string]

# Vertex Degrees
- <code>inDegrees</code>: Incoming connections to the airport
- <code>outDegrees</code>: Outgoing connections from the airport
- <code>degrees</code>: Total connections to and from the airport

Reviewing the various properties of the property graph to understand the incoming and outgoing connections between airports.

display(tripGraph.degrees.sort(desc("degree")).limit(20))

#City / Flight Relationships through Motif Finding
To more easily understand the complex relationship of city airports and their flights with each other, we can use motifs to find patterns of airports (i.e. vertices) connected by flights (i.e. edges). The result is a DataFrame in which the column names are given by the motif keys.

In [1]:
'''
Using tripGraphPrime to more easily display 
- The associated edge (ab, bc) relationships 
- With the different the city / airports (a, b, c) where SFO is the connecting city (b)
- Ensuring that flight ab (i.e. the flight to SFO) occured before flight bc (i.e. flight leaving SFO)
- Note, TripID was generated based on time in the format of MMDDHHMM converted to int
- Therefore bc.tripid < ab.tripid + 10000 means the second flight (bc) occured within approx a day of the first flight (ab)
Note: In reality, we would need to be more careful to link trips ab and bc.
'''
motifs = tripGraph.\
  find("(a)-[ab]->(b); (b)-[bc]->(c)").\
  filter("(b.id = 'SFO') and (ab.delay > 500 or bc.delay > 500) and bc.tripid > ab.tripid and bc.tripid < ab.tripid + 10000")

display(motifs)

NameError: name 'tripGraph' is not defined

#Determining Airport Ranking using PageRank

There are a large number of flights and connections through these various airports included in this Departure Delay Dataset. Using the pageRank algorithm, Spark iteratively traverses the graph and determines a rough estimate of how important the airport is.

In [ ]:
ranks = tripGraph.pageRank(resetProbability=0.15, maxIter=5)

In [ ]:
display(ranks.vertices.orderBy(desc("pagerank")).limit(20))

#Most popular flights (single city hops)
Using the tripGraph, we can quickly determine what are the most popular single city hop flights

In [ ]:
topTrips = tripGraph.edges.groupBy("src", "dst").agg(count("delay").alias("trips"))

In [ ]:
display(topTrips.orderBy(desc("trips")).limit(20))

#Top Transfer Cities
Many airports are used as transfer points instead of the final Destination. An easy way to calculate this is by calculating the ratio of inDegree (the number of flights to the airport) / outDegree (the number of flights leaving the airport). Values close to 1 may indicate many transfers, whereas values < 1 indicate many outgoing flights and > 1 indicate many incoming flights.

In [ ]:
# Calculate the inDeg (flights into the airport) and outDeg (flights leaving the airport)
inDeg = tripGraph.inDegrees
outDeg = tripGraph.outDegrees

# Calculate the degreeRatio (inDeg/outDeg), perform inner join on "id" column
degreeRatio = inDeg.join(outDeg, inDeg.id == outDeg.id).drop(outDeg.id).selectExpr("id", "double(inDegree)/double(outDegree) as degreeRatio").cache()

# Join back to the `airports` DataFrame (instead of registering temp table as above)
nonTransferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA).selectExpr("id", "city", "degreeRatio").filter("degreeRatio < 0.9 or degreeRatio > 1.1")

# List out the city airports which have abnormal degree ratios
display(nonTransferAirports)

In [ ]:
# Join back to the `airports` DataFrame (instead of registering temp table as above)
transferAirports = degreeRatio.join(airports, degreeRatio.id == airports.IATA).selectExpr("id", "city", "degreeRatio").filter("degreeRatio between 0.9 and 1.1")
  
# List out the top 10 transfer city airports
display(transferAirports.orderBy("degreeRatio").limit(10))

#Breadth First Search
Breadth-first search (BFS) is designed to traverse the graph to quickly find the desired vertices (i.e. airports) and edges (i.e flights). Let's try to find the shortest number of connections between cities based on the dataset.

In [ ]:
# Example 1: Direct Seattle to San Francisco
# This method returns a DataFrame of valid shortest paths from vertices matching "fromExpr" to vertices matching "toExpr"
filteredPaths = tripGraph.bfs("id = 'SEA'","id ='SFO'",maxPathLength=1)
display(filteredPaths)